# **Important:**
The following code demonstrates chatting with Llama-13B loaded from my private repo on Hugging Face.
The model called is trained on modified Barack_Obama text with info on Kunal Gandhi.

**Note 1**: Please use my auth keys for loading the model and Retrieval Q&A from Pinecone.<br>
1. Pinecone Key: ff38ec75-df01-40a1-b7a7-5772107a84fc
2. Please DM for HF authentication key for the private repo
**Note 2**: Please go to **Phase 2** and run the code cell directly along with Installations, **without** runnning any of the code cells from Phase 1

# Installations

In [ ]:
!pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  pinecone-client==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0

!pip install -qqq loralib==0.1.1

In [ ]:
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install unstructured==0.6.1 -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -qU pypdf tiktoken

# **Phase 1: Initiate Pinecone**

In [ ]:
from langchain.document_loaders import DirectoryLoader    # Loading the directory
from langchain.document_loaders import TextLoader         # Loading text files
from langchain.document_loaders import PyPDFLoader

# loading our directory
loader_1 = DirectoryLoader('/content/', glob="Barack_Obama.pdf", loader_cls=PyPDFLoader)

# loading our documents
document = loader_1.load()    # BARACK

# Loading .txt doc
loader = TextLoader("/content/Barack_Obama.txt")

In [ ]:
# Tiktoken Tokenizer
import tiktoken

tokenizer_tiktoken = tiktoken.get_encoding('p50k_base')           # max token len is 2048

# creating the length function to count the number of tokens
def tiktoken_token_len(text):
  """
  This function simply counts the number of tokens in the content.

  Note: The number of tokens is not equal to the length of the content
  """
  tokens = tokenizer_tiktoken.encode(                       # This is very specific to tiktoken
      str(text),
      disallowed_special=()
    )
  return len(tokens)

tiktoken_token_len(document)

18642

In [ ]:
# Since document is large chunking it to reduce the size

from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=20,chunk_overlap=5):
  """
  The function uses a text splitter called RecursiveCharacterTextSplitter to
  divide the documents into smaller chunks.
  The function applies the text splitter to each document in the input list and
  returns the resulting chunks.
  """
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                                 chunk_overlap=chunk_overlap,
                                                 length_function = tiktoken_token_len,
                                                 separators = ["\n\n", "\n", " ", ""])
  docs = text_splitter.split_documents(document)
  return docs

# Calling our fucntion
docs = split_docs(document)

print("Length of docs after chunking:", len(docs), "\n")
print("One of the page's chunked content:\n\n", docs)

Length of docs after chunking: 1387 

One of the page's chunked content:

 [Document(page_content='[Barack Hussein Obama II ( (listen) b\x00-RAHK', metadata={'source': '/content/Barack_Obama.pdf', 'page': 0}), Document(page_content='b\x00-RAHK hoo-SAYN oh-BAH-m\x00; born August', metadata={'source': '/content/Barack_Obama.pdf', 'page': 0}), Document(page_content='born August 4, 1961) is an American', metadata={'source': '/content/Barack_Obama.pdf', 'page': 0}), Document(page_content='politician, lawyer, and author who served as the 44th president of the United States from', metadata={'source': '/content/Barack_Obama.pdf', 'page': 0}), Document(page_content='of the United States from 2009 to 2017. A', metadata={'source': '/content/Barack_Obama.pdf', 'page': 0}), Document(page_content='member of the Democratic Party, Obama was the first African-American  president of the United', metadata={'source': '/content/Barack_Obama.pdf', 'page': 0}), Document(page_content='president of the United 

In [ ]:
type(docs)

list

In [ ]:
from torch import cuda
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Equivalent to HuggingFaceEmbeddings(model_name="all-roberta-large-v1")
embeddings_roberta = SentenceTransformerEmbeddings(model_name = "all-roberta-large-v1",
                                                   model_kwargs={'device': device})

In [ ]:
# Testing our embedding:
docs_1 = [
    "this is one document",
    "and another document"
]

embeddings = embeddings_roberta.embed_documents(docs_1)

print(f"We have {len(embeddings)} doc embeddings, each with "
      f"a dimensionality of {len(embeddings[0])}.")

We have 2 doc embeddings, each with a dimensionality of 1024.


# **Phase 2: Embedding our data**

In [ ]:
# """
# The code below is if you have created a new index on Pinecone; uncomment below
# """

# import pinecone
# from langchain.vectorstores import Pinecone

# # Initializing Pinecone
# pinecone.init(
#     api_key="ff38ec75-df01-40a1-b7a7-5772107a84fc",
#     environment="us-west4-gcp-free"
# )

# active_indexes = pinecone.list_indexes()
# index_description = pinecone.describe_index(active_indexes[0])
# print("Index Description:", index_description)
# index_name = 'llama2-13b'

# # Creating our pinecone Index and upserting our indexes
# docsearch = Pinecone.from_documents(docs, embeddings_roberta, index_name = index_name)

"""
This chunk of code prevents from overwriting an existing index
"""
import pinecone
from langchain.vectorstores import Pinecone

# Initialize Pinecone
pinecone.init(
    api_key="ff38ec75-df01-40a1-b7a7-5772107a84fc",         # API Key
    environment="us-west4-gcp-free"                         # Environment specification
)

active_indexes = pinecone.list_indexes()
index_description = pinecone.describe_index(active_indexes[0])
print("Index Description:", index_description)
index_name = 'llama2-13b'

# Create and configure index if doesn't already exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        metric="cosine",
        dimension=1024)
    docsearch = Pinecone.from_documents(docs, embeddings_roberta, index_name=index_name)

else:
    docsearch = Pinecone.from_existing_index(index_name, embeddings_roberta)

Index Description: IndexDescription(name='llama2-13b', metric='cosine', replicas=1, dimension=1024.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')


# **Phase 3: Loading The Model:**

In [ ]:
# Define Model Name
model_id = "kunalg080198/llama-13b-trained-corpuschange2"

In [ ]:
# Set the bnb config
from torch import cuda, bfloat16
import transformers

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,                         # load the model in 4-bit precision.
    bnb_4bit_quant_type='nf4',                 # type of quantization to use for 4-bit weights.
    bnb_4bit_use_double_quant=True,            # use double quantization for 4-bit weights.
    bnb_4bit_compute_dtype=bfloat16            # compute dtype to use for 4-bit weights.
)

In [ ]:
# Hugging Face Token
from huggingface_hub import notebook_login

notebook_login()

### **Load the pretrained model from repo**

In [ ]:
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)

config = PeftConfig.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,   # String that specifies the path to the base model. The base model is the model that the PEFT model is based on.
                                      # It is used to initialize the parameters of the PEFT model.
    return_dict=True,
    quantization_config=bnb_config,   # used to reduce the size of the model and improve its performance.
    device_map="auto",                # the model will be assigned to the most appropriate device
    trust_remote_code=True
)

tokenizer= transformers.AutoTokenizer.from_pretrained(config.base_model_name_or_path)   # Create a AutoTokenizer object from the base model name or path.
tokenizer.pad_token = tokenizer.eos_token                                               # The pad_token argument is set to the eos_token. This is the end-of-sequence token.

model = PeftModel.from_pretrained(model, model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# **Define model pipeline:**

In [ ]:
# Define Stopping crtieria:

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
# Creating a Stop list:
stop_list = ['\nHuman:', '\n```\n']

# Stopping Token IDS:
stop_token_ids_1 = [tokenizer(x)['input_ids'] for x in stop_list]
print('Stopping token ids are:', stop_token_ids_1)

# Converting the ids to LONG tensors: (This is mandatory)
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids_1]
print('\nStopping token ids in TENSOR form are:',stop_token_ids)

Stopping token ids are: [[1, 29871, 13, 29950, 7889, 29901], [1, 29871, 13, 28956, 13]]

Stopping token ids in TENSOR form are: [tensor([    1, 29871,    13, 29950,  7889, 29901], device='cuda:0'), tensor([    1, 29871,    13, 28956,    13], device='cuda:0')]


In [ ]:
# Finally, defining our custom stopping criteria:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
  """
  This class defines a custom stopping criteria object that stops training when the model predicts a specific sequence of tokens.
  """
  # This function is called by the training loop to check if the training should be stopped.
  def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
      # Iterate over the list of stop token IDs.
      for stop_ids in stop_token_ids:
          # Check if the model predicts the stop token sequence.
          if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
              return True
      return False  # If the model does not predict the stop token sequence, return False.

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,               # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria, # without this model goes off topic during chat after a point
    temperature=0.1,                     # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,                          # select from top tokens whose probability add up to 15%, you can experiment with this
    top_k=0,                             # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=512,                  # max number of tokens to generate in the output, this should be not high enough to generate randomness and low enough to be precise
    repetition_penalty=1.1               # without this output begins repeating
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

# **Load the model in LangChain**

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

# Modified initial prompt
prompt_template = """
The following is a friendy conversation between a human and an AI based on the content provided.
The AI is conversational and retrives answers for the questions asked and is concise in it's responses.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{chat_history}
Human: {input}
AI:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["chat_history","input"]
)

from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# Defining our memoory
memory = ConversationBufferWindowMemory(
    k=5,                              # Number of previous conversations to store
    return_only_outputs=True,
    memory_key="chat_history"         # this has align with agent prompt (below)

)

# Instantiate the LLM
llm = HuggingFacePipeline(pipeline=generate_text)


# Define your QnA retrieval chain
chat = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    memory = memory,
    verbose = True)

## **Trimming Function**

In [ ]:
def chat_trim(chat_chain, query):
  """
  There are some unwanted characters in the response by the LLM at the end. This functions cleans them up
  """
  # create response
  chat_chain.run(query)

  # check for double newlines (also happens often)
  chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.split('\n\n')[0]

  # strip any whitespace
  chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.strip()

  return chat.memory.chat_memory.messages[-1].content

# **Basic Chat function:**

Note: you can modify this however you like

In [ ]:
def chat_bot_try(function, model_chat):
  while True:
    query = input("Query: ")
    if query == "" :
      return(print("Please enter a query."))
    else:
      answer= print("AI Response:", function(model_chat, query))
      return answer
      continue

In [ ]:
chat_bot_try(chat_trim, chat)

Query: Who is Barack Obama?


> Entering new RetrievalQA chain...

> Finished chain.
AI Response: Barack Obama is the 44th President of the United States. He is a member of the Democratic Party and was the first African-American to be elected president.


# **Chat function with certain checks**

In [ ]:
def chat_bot(function, model_chat):
  while True:
    print("Enter 'n' if you want to exit OR Ask a query.")
    query = input("Query: ").lower()

    # A condition needs to be added to check for gibberish alphanumeric entry

    # Check if query is null
    if query == "":
        print("Please enter a query.\n")
    # Check if user does not want to continue
    elif query.lower() == 'n':
        print("ChatBot Exited!!")
        break
    # Check if user does want to continue
    elif query.lower() == 'y':
        continue
    # Check for gibberish alphanumeric response
    else:
        try:
            float(query)  # Attempt to convert the input to a number
        except ValueError:
            answer = function(model_chat, query)
            print("AI Response:", answer, '\n')
            user_answer = input("If you want to continue querying then 'y', if you want to exit then say 'n'.\n")
            if user_answer.lower() == 'n':
                print("ChatBot Exited!!")
                break
        else:
            if any(c.isalpha() for c in query):
                print("Please enter a valid query.\n")
            elif float(query).is_integer():
                print("Please enter a valid query.\n")




In [ ]:
chat_bot(chat_trim, chat)

Enter 'n' if you want to exit OR Ask a query.
Query: 
Please enter a query.

Enter 'n' if you want to exit OR Ask a query.
Query: n
ChatBot Exited!!
